## Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
    * Use external libraries `imutils`.
    * Modify the transformation matrix.

#### `imutils` library

In [11]:
pip install imutils


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software



In [12]:
import cv2 as cv
import imutils

# Load the image
image = cv.imread("all images/images/lena.jfif")

# Rotate the image by 45 degrees
rotated = imutils.rotate(image, 45)

# Display the original and rotated images
cv.imshow("Original Image", image)
cv.imshow("Rotated Image", rotated)

# Wait for a key press and close the windows
cv.waitKey(0)
cv.destroyAllWindows()

ModuleNotFoundError: No module named 'imutils'

#### Transformation Matrix

In [2]:
import cv2 as cv
import numpy as np

# Load the image
image = cv.imread("all images/images/lena.jfif")

# Get the dimensions of the image
(h, w) = image.shape[:2]

# Calculate the center of the image
center = (w // 2, h // 2)

# Create the rotation matrix
M = cv.getRotationMatrix2D(center, 45, 1.0)

# Calculate the sine and cosine of the rotation angle
abs_cos = abs(M[0, 0])
abs_sin = abs(M[0, 1])

# Compute the new bounding dimensions of the image
new_w = int((h * abs_sin) + (w * abs_cos))
new_h = int((h * abs_cos) + (w * abs_sin))

# Adjust the rotation matrix to take into account translation
M[0, 2] += new_w / 2 - center[0]
M[1, 2] += new_h / 2 - center[1]

# Perform the rotation and resize the image
rotated = cv.warpAffine(image, M, (new_w, new_h))

# Display the original and rotated images
cv.imshow("Original Image", image)
cv.imshow("Rotated Image", rotated)

# Wait for a key press and close the windows
cv.waitKey(0)
cv.destroyAllWindows()

2. Use the images with titles: _\"flower.jfif\"_ and _\"native-bee.png\"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following: 
   
   ![bee and flowers](img_embed/activity3.PNG)

In [4]:
# Load two images
img1 = cv.imread('images/native-bee.png')
img2 = cv.imread('images/flower.jfif')
assert img1 is not None, "file could not be read, check with os.path.exists()"
assert img2 is not None, "file could not be read, check with os.path.exists()"
 
# I want to put flower on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols]
 
# Now create a mask of logo and create its inverse mask also
img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2gray, 90, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)
 
# Now black-out the area of logo in ROI
img1_bg = cv.bitwise_and(roi,roi,mask = mask_inv)
 
# Take only region of logo from logo image.
img2_fg = cv.bitwise_and(img2,img2,mask = mask)
 
# Put logo in ROI and modify the main image
dst = cv.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst
 
cv.imshow('flower_bee',img1)
cv.waitKey(0)
cv.destroyAllWindows()

3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

In [7]:
import random
from utils import display_images

def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    """
    Randomly crops the central region of an image.
    
    Args:
        image (numpy.ndarray): The input image to be cropped.
        min_crop_ratio (float): The minimum crop ratio (0.0 to 1.0).
        max_crop_ratio (float): The maximum crop ratio (0.0 to 1.0).
    
    Returns:
        numpy.ndarray: The cropped image.
    """
    # Get the dimensions of the input image
    height, width = image.shape[:2]
    
    # Calculate the minimum and maximum crop sizes
    min_size = min(height, width)
    min_crop_size = int(min_size * min_crop_ratio)
    max_crop_size = int(min_size * max_crop_ratio)
    
    # Generate a random crop size within the specified range
    crop_size = random.randint(min_crop_size, max_crop_size)
    
    # Calculate the starting coordinates for the crop
    start_x = (width - crop_size) // 2
    start_y = (height - crop_size) // 2
    
    # Perform the center crop
    cropped_image = image[start_y:start_y+crop_size, start_x:start_x+crop_size]
    
    return cropped_image

In [9]:
# Load an image
image = cv.imread("images/meal.jpg")

# Set the minimum and maximum crop ratios
min_crop_ratio = 0.5
max_crop_ratio = 0.8

# Randomly crop the central region of the image
cropped_image = random_center_crop(image, min_crop_ratio, max_crop_ratio)

# Display the cropped image
display_images([image, cropped_image], ("original", "cropped image"))
cv.waitKey(0)
cv.destroyAllWindows()

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

#### Salt-and-Pepper noise

In [10]:
def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
    """
    Add salt and pepper noise to an image.
    
    Args:
        image (numpy.ndarray): The input image to which noise will be added.
        salt_prob (float): The probability of adding salt noise (white pixels).
        pepper_prob (float): The probability of adding pepper noise (black pixels).
    
    Returns:
        numpy.ndarray: The noisy image.
    """
    noisy_image = np.copy(image)
    
    # Get the dimensions of the image
    total_pixels = image.size
    num_salt = np.ceil(salt_prob * total_pixels)
    num_pepper = np.ceil(pepper_prob * total_pixels)

    # Add salt noise (white pixels)
    for _ in range(int(num_salt)):
        y = random.randint(0, image.shape[0] - 1)
        x = random.randint(0, image.shape[1] - 1)
        noisy_image[y, x] = 255  # Set pixel to white

    # Add pepper noise (black pixels)
    for _ in range(int(num_pepper)):
        y = random.randint(0, image.shape[0] - 1)
        x = random.randint(0, image.shape[1] - 1)
        noisy_image[y, x] = 0  # Set pixel to black

    return noisy_image

# Load an image
image = cv.imread('images/meal.jpg')

# Add salt and pepper noise to the image
salt_prob = 0.02  # 2% salt noise
pepper_prob = 0.02  # 2% pepper noise
noisy_image = add_salt_and_pepper_noise(image, salt_prob, pepper_prob)

# Display the original and noisy images
cv.imshow('Original Image', image)
cv.imshow('Salt and Pepper Noisy Image', noisy_image)

# Wait for a key press and close the windows
cv.waitKey(0)
cv.destroyAllWindows()